In [1]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    ORG,
    NamesExtractor,

    Doc
)

import pandas as pd
import numpy as np
from pprint import pprint

from ipymarkup import show_span_ascii_markup, show_dep_ascii_markup
from nerus import load_nerus


In [2]:
import spacy
from spacy.lang.ru import Russian
from spacy.lang.ru.examples import sentences

In [3]:
# reading the CSV file
csvFile = pd.read_csv('test_data.csv')


df = pd.DataFrame(data = csvFile)

df = df.to_numpy()

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

# I also wanted to add deeppavlov library due to unreachablilty of the repository I could not use it


In [4]:
mgrs_dialog = []
manager_dialog = []
cur = 0
for i in range(len(df)):
    if df[i][0] != cur:
        mgrs_dialog.append(manager_dialog)
        manager_dialog = []
        cur = df[i][0]
    if df[i][2] == 'manager':
        manager_dialog.append(df[i][3])

mgrs_dialog.append(manager_dialog)

In [5]:
from pullenti_wrapper.langs import (
    set_langs,
    RU
)
set_langs([RU])
from pullenti_wrapper.processor import (
    Processor,
    ORGANIZATION,
)
from pullenti_wrapper.referent import Referent


# I also wanted to use PullEnti library, but, after making some experiments, I found it not useful 

In [13]:
!python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 514.8 kB/s eta 0:00:00m eta 0:00:010:00:18
  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached pydantic-1.8.2-cp37-cp37m-manylinux2014_x86_64.whl (10.1 MB)
  Attempting uninstall: pymorphy2
    Found existing installation: pymorphy2 0.8
    Uninstalling pymorphy2-0.8:
      Successfully uninstalled pymorphy2-0.8
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.3
    Uninstalling pydantic-1.3:
      Successfully uninstalled pydantic-1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeppavlov 0.17.4 requires pydantic==1.3, but you have pydantic 1.8.2 which is incompatible.
deeppavlov 0.17.4 requires pymorphy2==0.8, but you have pymorphy2 0.9.1 which is incompatible.
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_

In [14]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01m
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [15]:
!python -m spacy download ru_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 624.3 kB/s eta 0:00:00m eta 0:00:010:00:03
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [11]:
import ru_core_news_lg
nlp = ru_core_news_lg.load()## for name ners

In [12]:
introduce = ['звать', 'мое имя', 'зовут']#, 'я', 'это']
grtngs = ['здравствуйте', 'привет', 'позвольте представиться',  'добрый день', 'добрый вечер',
          'доброе утро', 'утро доброе', 'вечер добрый','день добрый', 'приветствую']
greeting = False
byes = ['до свидания', 'всего хорошего', 'всего доброго']
bye = False

In [13]:
mgr_table = [['None' for x in range(6)] for y in range(len(mgrs_dialog))]


for j in range(len(mgrs_dialog)):
    greeting = False
    bye = False
    for i in range(len(mgrs_dialog[j])):
        text = mgrs_dialog[j][i]
        
        if i < 5:
            greeting = greeting or bool([c for c in grtngs if(c in text.lower())])
            if bool([c for c in grtngs if(c in text.lower())]):
                mgr_table[j][0] = text

            doc_spacy = nlp(text)
            for named_entity in doc_spacy.ents:
                if named_entity.label_ == PER and bool([c for c in introduce if(c in text.lower())]) and mgr_table[j][1] == 'None' and mgr_table[j][2] == 'None':
                    mgr_table[j][1] = text
                    mgr_table[j][2] = named_entity

            if 'компания' in text:
                company = ''
                doc_nat = Doc(text)
                doc_nat.segment(segmenter)
                for tok_pos in range(len(doc_nat.tokens)):
                    if doc_nat.tokens[tok_pos].text == 'компания':
                        break

                text = text[doc_nat.tokens[tok_pos].stop:]
                doc_nat = Doc(text)
                doc_nat.segment(segmenter)
                doc_nat.tag_morph(morph_tagger)

                pos = 0
                company = doc_nat.sents[0].morph.tokens[pos].text
                
                while doc_nat.sents[0].morph.tokens[pos].pos != 'NOUN':
                    pos +=1
                    company = company + ' ' + doc_nat.sents[0].morph.tokens[pos].text

                mgr_table[j][3] = company
        
        bye = bye or bool([c for c in byes if(c in text.lower())])
        if bool([c for c in byes if(c in text.lower())]):
            mgr_table[j][4] = text
    mgr_table[j][5] = greeting and bye

In [14]:
pprint(mgr_table)

[['Алло здравствуйте',
  'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления '
  'лицензии а мы с серым у вас скоро срок заканчивается',
  ангелина,
  'диджитал бизнес',
  'Всего хорошего до свидания',
  True],
 ['Алло здравствуйте',
  'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления '
  'а мы сели обратила внимание что у вас срок заканчивается',
  ангелина,
  'диджитал бизнес',
  'До свидания',
  True],
 ['Алло здравствуйте',
  'Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления '
  'лицензии а мастера мы с вами сотрудничали по видео там',
  ангелина,
  'диджитал бизнес',
  'None',
  False],
 ['Алло дмитрий добрый день',
  'Добрый меня максим зовут компания китобизнес удобно говорить',
  максим,
  'китобизнес',
  'Угу все хорошо да понедельника тогда всего доброго',
  True],
 ['None',
  'None',
  'None',
  'None',
  'Во вторник все ну с вами да тогда до вторника до свидания',
  False],
 ['None', 'None', 'None'